In [ ]:
from google.colab import files

# Upload the file from your local machine
uploaded = files.upload()

Ques 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

dataset = pd.read_csv('USA_Housing (1).csv')

X = dataset.drop('Price', axis=1)
y = dataset['Price']

print("Input features shape:", X.shape)
print("Output variable shape:", y.shape)



#scale, cleaning etc, then feature extr or corr, heatmap



#-----------Data transformation- scaling/normalization (mean method)
# Initialize the StandardScaler
scaler = StandardScaler()
# Fit and transform the input features
X_scaled = scaler.fit_transform(X)
# Convert the scaled features back to a DataFrame for convenience
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
print("Scaled features sample:")
print(X_scaled.head())


#---------------cleaning- missing values
from sklearn.impute import SimpleImputer
median_imputer = SimpleImputer(missing_values=np.nan, strategy="median")
# Fit the imputer on the training data
median_imputer.fit(X_scaled)
# Transform the training data
X_scaled = median_imputer.transform(X_scaled)



# Step 2: Cleaning the data (using z-score to remove outliers)
z_scores = np.abs(stats.zscore(X_scaled))
threshold = 3  # Defining the threshold for outliers
X_scaled_df = X_scaled[(z_scores < threshold).all(axis=1)]  # Keep rows with z-scores less than the threshold

# Ensure that shape of X_scaled_df is correct after filtering
#print("Shape after z-score filtering:", X_scaled_df.shape)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
# Step 3: Extreme value analysis (using IQR to remove outliers)
Q1 = X_scaled.quantile(0.25)
Q3 = X_scaled.quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 4: Remove outliers based on IQR
filtered_data = X_scaled[(X_scaled >= lower_bound) & (X_scaled <= upper_bound)].dropna()

# Ensure that the number of columns is consistent with the original dataset
print("Filtered data shape after outlier removal:", filtered_data.shape)

# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# Store R2 scores and beta coefficients for each fold
r2_scores = []
beta_coefficients = []

# Iterate through each fold
fold = 1
for train_index, test_index in kf.split(X_scaled):
    print(f"\nFold {fold}")

    # Split the data into training and testing sets
    X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Convert to numpy arrays
    X_train_np = X_train.values
    X_test_np = X_test.values
    y_train_np = y_train.values.reshape(-1, 1)
    y_test_np = y_test.values.reshape(-1, 1)

    # Add intercept term to X_train and X_test
    ones_train = np.ones((X_train_np.shape[0], 1))
    ones_test = np.ones((X_test_np.shape[0], 1))

    X_train_b = np.hstack([ones_train, X_train_np])
    X_test_b = np.hstack([ones_test, X_test_np])

    # Calculate beta coefficients using Normal Equation
    beta = np.linalg.inv(X_train_b.T.dot(X_train_b)).dot(X_train_b.T).dot(y_train_np)

    # Make predictions on the test set
    y_pred = X_test_b.dot(beta)

    # Calculate R2 score
    r2 = r2_score(y_test_np, y_pred)

    # Append results
    r2_scores.append(r2)
    beta_coefficients.append(beta)

    print(f"R2 Score: {r2}")

    fold += 1
# Find the index of the best R2 score
best_index = np.argmax(r2_scores)
best_beta = beta_coefficients[best_index]

print(f"\nBest R2 Score obtained in fold {best_index + 1}: {r2_scores[best_index]}")
print("Best Beta Coefficients:")
print(best_beta.flatten())

# Split the entire dataset into 70% training and 30% testing
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42)

# Convert to numpy arrays
X_train_full_np = X_train_full.values
X_test_full_np = X_test_full.values
y_train_full_np = y_train_full.values.reshape(-1, 1)
y_test_full_np = y_test_full.values.reshape(-1, 1)

# Add intercept term
ones_train_full = np.ones((X_train_full_np.shape[0], 1))
ones_test_full = np.ones((X_test_full_np.shape[0], 1))

X_train_full_b = np.hstack([ones_train_full, X_train_full_np])
X_test_full_b = np.hstack([ones_test_full, X_test_full_np])

# Make predictions on the test set using best beta
y_pred_full = X_test_full_b.dot(best_beta)

# Calculate R2 score
r2_full = r2_score(y_test_full_np, y_pred_full)

print(f"\nR2 Score on 30% test data using best beta: {r2_full}")

plt.figure(figsize=(10,6))
plt.scatter(y_test_full, y_pred_full, alpha=0.7, color='b')
plt.plot([y_test_full.min(), y_test_full.max()], [y_test_full.min(), y_test_full.max()], 'r--')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Actual vs Predicted Prices')
plt.show()


Ques 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('USA_Housing (1).csv')

# Split the dataset into input features (X) and output variable (y)
X = df.drop('Price', axis=1)
y = df['Price'].values.reshape(-1, 1)

# Step 1: Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 2: Add a column of ones to X_scaled (for the intercept term)
X_scaled = np.insert(X_scaled, 0, 1, axis=1)  # Adds the intercept (bias term)

# Step 3: Split dataset into train (56%), validation (14%), and test (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.44, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.68, random_state=42)

# Step 4: Gradient Descent function
def gradient_descent(X, y, alpha, num_iterations):
    m, n = X.shape
    beta = np.zeros((n, 1))  # Initialize regression coefficients (beta) to zeros
    cost_history = []

    for i in range(num_iterations):
        predictions = X.dot(beta)
        error = predictions - y
        cost = (1/(2*m)) * np.sum(error ** 2)
        cost_history.append(cost)
        gradient = (1/m) * X.T.dot(error)  # Gradient
        beta = beta - alpha * gradient  # Update beta

    return beta, cost_history

# Step 5: Training the model for different learning rates
learning_rates = [0.001, 0.01, 0.1, 1]
num_iterations = 1000
best_r2_val = -np.inf
best_beta = None
best_lr = None

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")
    beta, cost_history = gradient_descent(X_train, y_train, alpha=lr, num_iterations=num_iterations)

    # Step 6: Compute R² score on validation set
    val_predictions = X_val.dot(beta)
    r2_val = r2_score(y_val, val_predictions)

    # Print the R² score for validation set
    print(f"R² score on validation set: {r2_val}")

    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_beta = beta
        best_lr = lr

# Step 7: Final evaluation on test set using the best beta
test_predictions = X_test.dot(best_beta)
final_r2_test = r2_score(y_test, test_predictions)

print(f"\nBest learning rate: {best_lr}")
print(f"Best R² score on validation set: {best_r2_val}")
print(f"Final R² score on test set: {final_r2_test}")

# Optional: Plot cost history to visualize convergence
#import matplotlib.pyplot as plt

#plt.plot(cost_history)
#plt.title(f"Cost Reduction over Iterations (Learning Rate = {best_lr})")
#plt.xlabel("Iterations")
#plt.ylabel("Cost (Mean Squared Error)")
#plt.show()


Ques 3

In [ ]:
import numpy as np
import pandas as pd

colnames= ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",
            "engine_location", "wheel_base", "length", "width", "height", "curb_weight",
            "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
            "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]
df=pd.read_csv('imports-85.data.txt',header=None, names=colnames, na_values="?" )

df=df.replace('?',np.NaN)
df.head()

from sklearn.impute import SimpleImputer
df.dropna(subset=['price'],inplace=True)
imputer = SimpleImputer(missing_values = np.nan,strategy ='most_frequent') #replace NAN with mode
imputer1=imputer.fit(df)
df=pd.DataFrame(imputer1.transform(df)) #pd.DataFrame is required becaude by default imputer returns an ndarray
df.columns=colnames #specifying column names
print(df)

dict1={"one":1,"two":2,"three":3,"four":4,"five":5,"six":6,"seven":7,"eight":8,"nine":9,"zero":0,"twelve":12} #mapping the values
df['num_doors']=df['num_doors'].map(dict1)
df['num_cylinders']=df['num_cylinders'].map(dict1)
print(df)

#dummy encoding scheme
dummy_encoding_body_style=pd.get_dummies(df['body_style'],prefix="body_style",drop_first=True)
df=pd.concat([df,dummy_encoding_body_style],axis=1)
df.drop(['body_style'],axis=1,inplace=True)
dummy_encoding_drive_wheels=pd.get_dummies(df['drive_wheels'],prefix="drive_wheels",drop_first=True)
df.drop(['drive_wheels'],axis=1,inplace=True)
df=pd.concat([df,dummy_encoding_drive_wheels],axis=1)
print(df)

#label encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['make']=encoder.fit_transform(df['make'])
df['aspiration']=encoder.fit_transform(df['aspiration'])
df['engine_location']=encoder.fit_transform(df['engine_location'])
df['fuel_type']=encoder.fit_transform(df['fuel_type'])
df.head()

cond=(df['fuel_system'].str.contains('pfi')==True)
df.loc[cond,'fuel_system']=1
cond2=(df['fuel_system']).str.contains('pfi')==False
df.loc[cond2,'fuel_system']=0

cond3=(df['engine_type'].str.contains('ohc')==True)
cond4=((df['engine_type']).str.contains('ohc')==False)
df.loc[cond3,'engine_type']=1
df.loc[cond4,'engine_type']=0

df.head()
df.info()

x_data=df.loc[:,df.columns!='price']
y_data=df.loc[:,df.columns=='price']

x_data.info()
y_data.info()

#Scaling the input data
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
features_to_scale=x_data.select_dtypes(np.number).columns
# print(df.columns)
x_scaled=scaler.fit_transform(x_data[features_to_scale]) #specifying the columns to normalise
x_Scaled=pd.DataFrame(x_scaled,columns=features_to_scale)
x_data=x_Scaled #assigning the scaled values to x_data
print(x_data)

#Train a linear regressor on 70% of data (using inbuilt linear regression function of Python) and test its performance on remaining 30% of data.
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3,random_state=42)
lr=LinearRegression()
lr.fit(x_train,y_train)
print(lr.score(x_test,y_test))

#Dimensional Reduction using PCA
# Check the number of features after processing
print(f"Number of features after preprocessing: {x_data.shape[1]}")

# Determine the maximum number of components for PCA
max_components = x_data.shape[1]  # This should give you the number of features
print(f"Maximum components for PCA: {max_components}")

# Dimensional Reduction using PCA
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Set n_components to the minimum of 6 or max_components
n_components = min(max_components, 6)  # Adjust to the number of features available
print(f"Using {n_components} components for PCA")

pca = PCA(n_components=n_components)
x_pca = pca.fit_transform(x_data)

# Split the PCA transformed data into training and testing sets
x_train_pca, x_test_pca, y_train_pca, y_test_pca = train_test_split(x_pca, y_data, test_size=0.3, random_state=42)

# Train a linear regressor on PCA transformed data
lr_pca = LinearRegression()
lr_pca.fit(x_train_pca, y_train_pca)

# Evaluate the model
pca_score = lr_pca.score(x_test_pca, y_test_pca)
print(f"R² Score after PCA: {pca_score:.4f}")

